# Sentinel-1 Cumulative Sum Analysis to Map Forest Disturbances

__Author__: Chiara Aquino <br>
__Date__: 30 Aug 2022

Output: 2 GeoTiff rasters:
- Intensity of forest disturbance (as peak of the CuSum curve, in $\gamma_{0}$ units)
- Date of forest disturbance (in Date Of Year units)



In [6]:
import ee
ee.Initialize()

import geemap
import os
import xarray as xr
import rioxarray

#### Image Processing Functions

In [ ]:
#convert from sigma_0 to gamma_0
def toGamma0(image):
  gamma0 = image.select('VV').subtract(image.select('angle').multiply(np.pi/180.0).cos().log10().multiply(10.0));
  return gamma0.copyProperties(image).copyProperties(image,['system:time_start']);

# mosaic images with the same date that have been spatially split
def mosaicByDate(imcol):
    imlist = imcol.toList(imcol.size())
    unique_dates = imlist.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")).distinct()
    def mosaic_imlist(d):
        d = ee.Date(d)
        im = imcol.filterDate(d, d.advance(1, "day")).mosaic() 
        return im.set(
        "system:time_start", d.millis(), 
        "system:id", d.format("YYYY-MM-dd"));

    return ee.ImageCollection(unique_dates.map(mosaic_imlist))

#### Define input parameters 

In [4]:
# define year of the analysis

year = 2020;
START_DATE = str(year-1)+"-01-01"; #no image before
END_DATE = str(year+1)+"-07-01"; #exclusive = ie., until 2021-06-31

# define image projection 

crs = "EPSG:4326"

In [ ]:
# define your area of interest

aoi_shp = 'path/to/shapefile_with_aoi'
aoi = geemap.shp_to_ee(aoi_shp)

# choose a path for your output directory. This is where you will have saved all the images in the collection

outdir = 'path/to/your/ouput_directory_image_collection/'

#### Download Image Collection


In [ ]:
collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
          .filterBounds(region).filterDate(ee.Date(START_DATE), ee.Date(END_DATE))
          .filter(ee.Filter.eq('instrumentMode', 'IW'))
          .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
          .filterMetadata('transmitterReceiverPolarisation', 'equals', ['VV', 'VH'])
          .filterMetadata('resolution_meters', 'equals', 10)
          .map(toGamma0)
          .map(lambda image:image.clip(aoi.geometry())))

# select VV polarization
vv = collection.select(['VV']) 
vv_size = vv.size().getInfo()
print('original size of Image Stack: ', vv_size)

#get image dates
imlist = vv.toList(vv.size())
unique_dates = imlist.map(lambda im:ee.Image(im).date().format("YYYY-MM-dd")).distinct()
date_list = unique_dates.getInfo()

#mosaic images with same date
vv_mosaic = mosaicByDate(vv)
mosaic_size = vv_mosaic.size().getInfo()
print('size of Image Stack after mosaicking: ' ,mosaic_size)

#export Image Collection to your folder
geemap.ee_export_image_collection(vv_mosaic, scale=10,out_dir=outdir, region=aoi.geometry(),crs=crs)
        

#### Convert Image Collection to xarray

In [ ]:
# list all downloaded images
vv_tifs = os.listdir(outdir)
files = list(filter(os.path.isfile, glob.glob(outdir + "*.tif")))
    
#sort images by download time
files.sort(key=lambda x: os.path.getctime(x))

# loop through the list, open image as xarray and assign time label
list_da=[]
for file, date in zip(files, date_list):
    da = xr.open_rasterio(file,masked=True)
    dt = datetime.datetime.strptime(date,"%Y-%m-%d")
    dt = pd.to_datetime(dt)
    da = da.assign_coords(time = dt)
    da = da.expand_dims(dim="time")
    list_da.append(da)
    
#stack data arrays in list
ds = xr.combine_by_coords(list_da)


#### CumSum implementation

In [ ]:
# get timeseries mean
dsmean = ds.mean(dim='time')
#get time series residual
R = ds-dsmean 
# get time series cumulative sum
S = R.cumsum(dim="time") 
# get maximum of the cumulative sum
Smax= S.max(dim="time")     
# the threshold is calculated as 99th percentile of the CuSum max
threshold = np.percentile(Smax, 99) 
# filter cumulative sum array by year of interest
Sfilt_time = S.sel(time=year) 
# convert to DOY
Sfilt_time['time'] = Sfilt_time["time.dayofyear"]
# spatially filter by 99th percentile
Sfilt_n = Sfilt_time.where(Sfilt_time>= threshold,np.nan)
# determine where you have valid data
mask = Sfilt_n['time'].isel(time=0).notnull()  
#convert Nan to calculate maximum
Sfilt_n2 = Sfilt_n.fillna(-9999)
# get the date where the curve reaches the maximum value
Sfilt_max = Sfilt_n2.isel(time = Sfilt_n2.argmax('time')).where(mask) 

max_values = Sfilt_max.where(Sfilt_max> -9999,np.nan)
max_dates = Sfilt_n.idxmax(dim="time")

max_values.name = 'Smax'
max_dates.name = 'doy'

#### Save outputs


In [ ]:
intensityName = 'path/output/folder/Smax.tif'   # path to your folder for Smax intensity image
dateName = 'path/output/folder/Dates.tif'       # path to your folder for Date image

max_values_raster = max_values.rio.write_crs(crs)
max_values_raster.rio.to_raster(intensityName,compress='LZMA')

max_dates_raster = max_dates.rio.write_crs(crs)
max_dates_raster.rio.to_raster(dateName,compress='LZMA')